In [2]:
IRIS = r'C:\Users\mr068144\Downloads\IRIS\IDOT Zip Files\HWY2016_route.shp'
Path = r'C:\Users\mr068144\Downloads\Curvature'
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
# importing required packages
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy
import arcpy
import shutil
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
GDB = os.path.join(Path,'Scratch.gdb')
IRIS_N = GDB + '\\HorCurves2016'

In [5]:
if os.path.exists(os.path.join(Path,'Scratch.gdb')):
    shutil.rmtree(os.path.join(Path,'Scratch.gdb'))
arcpy.CreateFileGDB_management(out_folder_path=Path,out_name='Scratch')

<Result 'C:\\Users\\mr068144\\Downloads\\Curvature\\Scratch.gdb'>

In [6]:
def progressBar(value, endvalue, bar_length=20):
        percent = float(value) / endvalue
        sys.stdout.write("\r{}%".format(int(round(percent * 100))))
        sys.stdout.flush()

In [7]:
def AddMidPoints(l,desd):
    pntl = [arcpy.Point(X = l[0][0],Y=l[0][1],Z=0,M=l[0][3])]
    for p in l[1:]:
        curpnt = arcpy.Point(X = p[0],Y=p[1],Z=0,M=p[3])
        curd = arcpy.PointGeometry(pntl[-1]).distanceTo(curpnt)
        if curd <= desd:
            pntl.append(curpnt)
        else:
            n = int(curd/desd)+1
            delta = curd/n
            pl = arcpy.Polyline(arcpy.Array([pntl[-1],curpnt]))
            for j in range(1,n):
                if j*delta<curd:
                    midpg = pl.positionAlongLine(j*delta)
                    m = float((curpnt.M - pntl[-1].M))/n*j + pntl[-1].M
                    midp = arcpy.Point(midpg.firstPoint.X,midpg.firstPoint.Y,0,m)
                    pntl.append(midp)
            pntl.append(curpnt)
    return(pntl)

In [8]:
def FindCurves(pl,DegMin,RMax,RMin,LenMin):
    import json
    Curve = []
    pntl = json.loads(pl.JSON)['paths'][0]
    CD = HorCurvature(pl)
    flag = False
    for i in range(len(CD['Radius'])):
        if abs(CD['Radius'][i]) > DegMin:
            if not flag:
                start = CD['Milepost'][i]
                flag = True
                R = [pntl[i-1],pntl[i]]
            else:
                if CD['Radius'][i]*CD['Radius'][i-1]>0:
                    R.append(pntl[i])
                else:
                    end = (CD['Milepost'][i-1] + CD['Milepost'][i])/2.0
                    R.append(pntl[i])
                    cirD = CircleFitting(R)
                    cirD['Radius'] = math.copysign(cirD['Radius'],CD['Radius'][i-1])
                    if abs(cirD['Radius'])<RMax and abs(cirD['Radius'])>RMin and (end-start)>LenMin:
                        Curve.append([start,end,cirD['Radius'],cirD['Center'][0],cirD['Center'][1]])
                    start = end
                    flag = True
                    R = [pntl[i-1],pntl[i]]
        else:
            if flag:
                end = CD['Milepost'][i]
                R.append(pntl[i])
                cirD = CircleFitting(R)
                cirD['Radius'] = math.copysign(cirD['Radius'],CD['Radius'][i-1])
                if abs(cirD['Radius'])<RMax and abs(cirD['Radius'])>RMin and (end-start)>LenMin:
                    Curve.append([start,end,cirD['Radius'],cirD['Center'][0],cirD['Center'][1]])
                flag = False
    return(Curve)

In [9]:
def CircleFitting(l):
    from scipy import optimize
    import numpy
    def calc_R(xc, yc):
        return numpy.sqrt((x-xc)**2 + (y-yc)**2)
    def f_2(c):
        Ri = calc_R(*c)
        return Ri - Ri.mean()
    x = numpy.array([i[0] for i in l])
    y = numpy.array([i[1] for i in l])
    x_m = sum(x)/max(len(x),1)
    y_m = sum(y)/max(len(y),1)
    center_estimate = x_m, y_m
    center_2, ier = optimize.leastsq(f_2, center_estimate)
    xc_2, yc_2 = center_2
    Ri_2       = calc_R(*center_2)
    R_2        = Ri_2.mean()
    residu_2   = sum((Ri_2 - R_2)**2)
    return({'Radius':R_2,'Center':[xc_2, yc_2]})

In [10]:
def HorCurvature(Shape):
    import re
    import math
    import arcpy
    from math import acos
    from numpy.linalg import norm
    import numpy
    import json
    Inf = 52800 
    def findangle(p1, p2,p3):
        A = np.array(p1)
        B = np.array(p2)
        C = np.array(p3)
        v1 = B - A
        v2 = C - B
        def unit_vector(vector):
            return vector / np.linalg.norm(vector)
        v1_u = unit_vector(v1)
        v2_u = unit_vector(v2)
        return(np.degrees(np.arctan2(v2_u[1], v2_u[0])-np.arctan2(v1_u[1], v1_u[0])))
    def Radius(P1,P2,P3):
        Inf = 52800
        x1 = P1[0];x2 = P2[0];x3 = P3[0]
        y1 = P1[1];y2 = P2[1];y3 = P3[1]
        if y1==y2 and y2==y3:
            R = Inf
        elif y1==y2 and y2<>y3:
            m2 = -(x3-x2)/(y3-y2)
            xm1 = (x1+x2)/2
            ym1 = (y1+y2)/2
            xm2 = (x3+x2)/2
            ym2 = (y3+y2)/2
            xc = xm1
            yc = m2*(xc-xm2)+ym2
            R  = math.sqrt((xc-x1)**2+(yc-y1)**2)
            R = min(R,Inf)
            R = math.copysign(R,m2)
        elif y2==y3 and y1<>y2:
            m1 = -(x2-x1)/(y2-y1)
            xm1 = (x1+x2)/2
            ym1 = (y1+y2)/2
            xm2 = (x3+x2)/2
            ym2 = (y3+y2)/2
            xc = xm2
            yc = m1*(xc-xm1)+ym1
            R  = math.sqrt((xc-x1)**2+(yc-y1)**2)
            R = min(R,Inf)
            R = math.copysign(R,-m1)
        elif y1<>y2 and y3<>y2:
            if y3 == y1:
                R = Inf
            elif y3<>y1:
                if (x3-x1)/(y3-y1) == (x2-x1)/(y2-y1):
                    R = Inf
                else:
                    m1 = -(x2-x1)/(y2-y1)
                    m2 = -(x3-x2)/(y3-y2)
                    xm1 = (x1+x2)/2
                    ym1 = (y1+y2)/2
                    xm2 = (x3+x2)/2
                    ym2 = (y3+y2)/2
                    xc = (ym1-ym2+m2*xm2-m1*xm1)/(m2-m1)
                    yc = m1*(xc-xm1)+ym1
                    R  = math.sqrt((xc-x1)**2+(yc-y1)**2)
                    R = min(R,Inf)
                    R = math.copysign(R,m2-m1)
        return(min(R,Inf))
    def Length(P1,P2):
        return(math.sqrt((P2[0]-P1[0])**2+(P2[1]-P1[1])**2))
    Vertices = json.loads(Shape.JSON)['paths'][0]
    R = [0]
    M = [Vertices[0][3]]
    L = Shape.length
    for i in range(2,len(Vertices)):
        l1 = Length(Vertices[i-2],Vertices[i-1])
        l2 = Length(Vertices[i-1],Vertices[i  ])
        #R.append(Radius(Vertices[i-2],Vertices[i-1],Vertices[i]))
        R.append(findangle(Vertices[i-2],Vertices[i-1],Vertices[i]))
        M.append(Vertices[i-1][3])
    R.append(0)
    M.append(Vertices[-1][3])
    return({'Radius':R,'Milepost':M})

In [12]:
arcpy.Delete_management(IRIS_N)
arcpy.CreateTable_management(out_path=os.path.dirname(IRIS_N),out_name=os.path.basename(IRIS_N))
arcpy.AddField_management(IRIS_N,'INVENTORY','TEXT')
arcpy.AddField_management(IRIS_N,'BEG_STA','DOUBLE')
arcpy.AddField_management(IRIS_N,'END_STA','DOUBLE')
arcpy.AddField_management(IRIS_N,'Radius','DOUBLE')
arcpy.AddField_management(IRIS_N,'Center_X','DOUBLE')
arcpy.AddField_management(IRIS_N,'Center_Y','DOUBLE')
INV = {r.getValue('INVENTORY'):r.getValue('Shape') for r in arcpy.SearchCursor(IRIS)}
IC = arcpy.InsertCursor(IRIS_N)
desd = 150
for inv in INV.keys():
    l = json.loads(INV[inv].JSON)['paths'][0]
    if not None in [p[3] for p in l]:
        pntl = AddMidPoints(l,desd) 
        a = arcpy.Array(pntl)
        pl = arcpy.Polyline(a,arcpy.SpatialReference(102672),True,True)
        Curve = FindCurves(pl,DegMin=3,RMax=900,RMin=50,LenMin=100/5280)
        for cur in Curve:
            r = IC.newRow()
            r.setValue('INVENTORY',inv)
            r.setValue('BEG_STA',cur[0])
            r.setValue('END_STA',cur[1])
            r.setValue('Radius',cur[2])
            r.setValue('Center_X',cur[3])
            r.setValue('Center_Y',cur[4])
            IC.insertRow(r) 
    progressBar(INV.keys().index(inv),len(INV))
del IC
del r

100%

In [4]:
CHCurves = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\8.0_Curves_Layer\Extracting_Curves_From_Routes\HCurves2016.shp'
IDOTCurves = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\8.0_Curves_Layer\IL_SegmentCurves.mdb\Curves_Filtered_CH2M_Additions'
l1 = [r.getValue('Shape').length/5280 for r in arcpy.SearchCursor(CHCurves)]
l2 = [r.getValue('Shape').length/5280 for r in arcpy.SearchCursor(IDOTCurves)]
r1 = [abs(r.getValue('Radius')) for r in arcpy.SearchCursor(CHCurves)]
r2 = [r.getValue('avgRadius') for r in arcpy.SearchCursor(IDOTCurves)]
print('Our Curve Layer: Total Length {:0,.0f}, Average '.format(sum(l1)))
print(sum(l2))

Our Curve Layer: Total Length 13,260, Average 
13008.3790065


In [5]:
sum(r1)/len(r1)

379.49818698040275

In [6]:
sum(r2)/len(r2)

566.4342708407613